# Integrate SerpApi + Gemini Social media post using Langchain tool

### Import libraries and api keys

In [1]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.schema import SystemMessage, HumanMessage
from serpapi import GoogleSearch
from dotenv import load_dotenv
from IPython.display import display, Markdown

load_dotenv()

SERPAPI_API_KEY = os.getenv('SERPAPI_API_KEY')
if SERPAPI_API_KEY is None:
  raise Exception('Missing Serpapi api key')

GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')
if GEMINI_API_KEY is None:
  raise Exception('Missing Gemini api key')

### Setup gemini with langchain

In [14]:
llm = ChatGoogleGenerativeAI(
  model = 'models/gemini-1.5-flash',
  temperature = 0.7,
  google_api_key = GEMINI_API_KEY
)

### Fetch news using SerpApi

In [3]:
def fetch_ai_news_serpapi(query = "latest AI news July 2025", num_results = 5):
  params = {
    "q": query,
    "location": "United Kingdom",
    "hl": "en",
    "gl": "gb",
    "api_key": SERPAPI_API_KEY,
    "num": num_results,
    "engine": "google"
  }

  search = GoogleSearch(params)
  results = search.get_dict()

  return results.get("organic_results", [])[:num_results]

### Langchain prompt template

In [5]:
prompt_template = PromptTemplate(
  input_variables=["news_text"],
  template="""
  You are an expert in AI communication and social media perception.

  Given the following AI-related news articles:

  {news_text}

  For each:
  - Summarize the core topic in 1 sentence.
  - Say in 1 sentence whether this is effective for LinkedIn and why or why not.
  - Give a rating out of 10 for LinkedIn impact.

  Finally, recommend the top 2 stories to post on LinkedIn and explain why.
  """
)

### Pipeline Execution

In [10]:
def run_pipeline():
  news_items = fetch_ai_news_serpapi()
  
  if not news_items:
    print("No news found")
    return
  
  formatted = ""
  for i, item in enumerate(news_items, 1):
    title = item.get("title", "")
    snippet = item.get("snippet", "")
    link = item.get("link", "")
    formatted += f"{i}. {title}\n{snippet}\nURL: {link}\n\n"

  print(formatted)

  prompt = prompt_template.format(news_text = formatted)

  response = llm.invoke([
    SystemMessage(content="You are a helpful assistant summarizing and analyzing tech news."),
    HumanMessage(content=prompt)
  ])

  print(response.content)
  

### Run the LangChain + Gemini + SerpAPI news agent

In [15]:
run_pipeline()

1. AI News | Latest AI News, Analysis & Events
AGI. July 17, 2025. Apple loses key AI leader to Meta · Apple. July 8, 2025. Flood of interest in Europe's AI Gigafactories plan · Artificial Intelligence. July ...
URL: https://www.artificialintelligence-news.com/

2. The latest AI news we announced in June
Here's a recap of some of our biggest AI updates from June, including more ways to search with AI Mode, a new way to share your NotebookLM notebooks publicly.
URL: https://blog.google/technology/ai/google-ai-updates-june-2025/

3. Latest AI Breakthroughs and News: May, June, July 2025
Summary: OpenAI unveiled GPT-4.5, the newest iteration of its AI language model, focusing heavily on improved emotional intelligence (EQ). The upgrade ...
URL: https://www.crescendo.ai/news/latest-ai-news-and-updates

4. Artificial intelligence (AI)
17 July 2025 · OpenAI launches personal assistant capable of controlling files and web browsers · UK switches on AI supercomputer that will help spot sick cow